In [3]:
import numpy as np
import math
import pandas as pd
import hashlib, random
from sklearn.mixture import GaussianMixture
import copy
import itertools
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
import jsonpickle
from random import shuffle
from collections import defaultdict
import os, json
from datetime import datetime
import matplotlib.ticker as mticker
from matplotlib.ticker import MaxNLocator
pd.options.display.float_format = '{:,.2f}'.format
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import random, json
from tqdm import trange, tqdm
from operator import itemgetter
from analyzer import *
import query_gen

sns.set(style='ticks', palette='coolwarm') #this overwrites matplotlib setting so make it run first 

# def set_font():
import matplotlib
# http://ishxiao.com/blog/python/2017/07/23/how-to-change-the-font-size-on-a-matplotlib-plot.html

font = {'weight' : 'bold',
#         'size'   : 20,
       'family': 'Linux Libertine O'}
matplotlib.rc('font', **font)

/local/xi/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
def analyze_ratio(results):
    cnt_mean_ratios = {}
    sum_mean_ratios = {}
    cnt_median_ratios = {}
    sum_median_ratios = {}
    cnt_ratios = {}
    sum_ratios = {}
    gt = results['GT']
    for name, result in results.items():
        if name == 'GT':
            continue
#         print(name)
#         print(gt)
#         print(result)
        cnt_ratio = [float(b[0])/float(g[0]) for b, g in zip(result, gt)]
        cnt_mean_ratios[name] = np.mean(cnt_ratio)
        cnt_median_ratios[name] = np.median(cnt_ratio)
#         print(pd.DataFrame(cnt_ratio).describe())
        sum_ratio = [float(b[1])/float(g[1]) for b, g in zip(result, gt)]
        sum_mean_ratios[name] = np.mean(sum_ratio)
        sum_median_ratios[name] = np.median(sum_ratio)
        sum_ratios[name] = sum_ratio
        cnt_ratios[name] = cnt_ratio
    return cnt_ratios, sum_ratios, cnt_mean_ratios, cnt_median_ratios, sum_mean_ratios, sum_median_ratios

def plot_columns(mean, median, ylabel='Bound/GroundTruth Ratio', 
                 xlabel='Mean and Median', bar_labels=['Mean Ratio', 'Median Ratio'],
                log_scale = True, yrange= None):
    patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*" ]
    baselines = mean.keys()
#     print(baselines)
    if yrange is None:
        yrange = [0, max(max(mean.values()), max(median.values())) + 1]
    fig, ax = plt.subplots(1,1,figsize=(18,8))
    
    xfont = {'weight' : 'bold',
            'size'   : 22}

    width = 0.3
    
    mean_bar = [perf for perf in mean.values()]
    median_bar = [perf for perf in median.values()]

    r0 = np.arange(len(mean_bar))
    r1 = [x - 0.15 for x in r0]
    r2 = [x + width for x in r1]
#     print(mean_bar, median_bar)
    ax.bar(r1, mean_bar, width, edgecolor='black', label=bar_labels[0], hatch=patterns[0])
    ax.bar(r2, median_bar, width, edgecolor='black', label=bar_labels[1], hatch=patterns[1])
    ax.set_xticks(np.arange(len(baselines)))
    ax.set_xticklabels(baselines, fontsize='x-large')
    
    ax.hlines(1.0, -0.5, len(baselines)-0.5, linestyle='--', color='gray', linewidth=4, label='Ground Truth')
    
    ax.set_ylabel(ylabel, fontdict = xfont)
    
    ax.set_ylim(yrange)
    
    if log_scale:
        ax.set_yscale('log')
        
    ax.legend()
    
#     matplotlib.pyplot.sca(ax)
    print(yrange)

    plt.legend(fontsize=20, loc='upper left')
    ax.set_xlabel(xlabel, fontdict = xfont)
    fig.tight_layout()
    plt.show()

    
# https://seaborn.pydata.org/tutorial/color_palettes.html
def plot_box(cnt_ratios, sum_ratios, ylabel='Bound/GroundTruth Ratio', xlabel='Mean and Median', bar_labels=['COUNT(*)', 'SUM(Light)']):
    patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*" ]
    baselines = cnt_ratios.keys()
#     print(max([max(r) for r in cnt_ratios.values()]), max(cnt_ratios.values()))
#     yrange = [0, max(max([max(r) for r in cnt_ratios.values()]), max([max(r) for r in sum_ratios.values()])) + 1]
    fig, ax = plt.subplots(1,1,figsize=(18,8))
    xfont = {'weight' : 'bold',
            'size'   : 22}

    width = 0.5
    
    cnt_box = [perf for perf in cnt_ratios.values()]
    sum_box = [perf for perf in sum_ratios.values()]

    r0 = np.arange(len(cnt_ratios))
    r1 = [x - 0.15 for x in r0]
    r2 = [x + width for x in r1]
#     print(mean_bar, median_bar)
    ax.boxplot(cnt_box, positions = r1, showfliers=False)
    ax.boxplot(sum_box, positions = r2, showfliers=False)
    ax.set_xticks(np.arange(len(baselines)+1))
    ax.set_xticklabels(baselines, fontsize='x-large')

    ax.hlines(1.0, -1, len(baselines)+1.5, linestyle='--', color='gray', linewidth=2, label='Ground Truth')

    ax.set_ylabel(ylabel, fontdict = xfont)
    ax.set_xlim([-1, len(baselines)+1])
#     ax.set_ylim(yrange)
#         ax.set_yscale("log")

    ax.legend()

    #     matplotlib.pyplot.sca(ax)

#         plt.legend(fontsize=20, loc='upper left')
#     ax.set_xlabel(xlabel, fontdict = xfont)
    fig.tight_layout()
    plt.show()

In [5]:


def restore_obj(path):
    try:
        with open(path) as f:
            s = f.read()
            data = jsonpickle.decode(s, keys = True)
            print("Loaded %s" % path)
            return data
    except Exception as e:
        print("failed to restore", path, e)
        return None

def load_and_merge(config):
    # load all the files and merge them together because we have different files representing different partition type.
    baseline_perf = defaultdict(dict)
    queries = {}
#     print(config)
    for name, path in config.items():
        path = '/local/xi/VarAcc/src/cache/'+path
        if 'json' in path:
            data = json.load(open(path))
        else:
            data = restore_obj(path)
        if 'GT' in name:
            queries[name[-1]] = data
        else:
            k = name.split('-')[0]
            for base, perf in data.items():
                baseline_perf[k][base] = perf
    return queries, baseline_perf


In [6]:
!hostname

vulture


In [9]:
ds_name = "Taxi"

n_queries = {'random': 1000}
     
srs=[0.005]

nds = [1,2,3,4,5]

baselines= ["KD-PASS"]
bp={}

result_paths={
'intel' : 'taxi-verdict-intel.json',
'insta' : 'taxi-verdict-insta.json',
'taxi' : 'taxi-verdict-taxi.json',
}
# Taxi3D-MedianPartitioner-AQP++-5-1024-0.005-1000-random.json
name_temp = "{nd}-{alias}"

baselines= ["intel", 'insta', 'taxi']
vconfig={}
for workload, n_query in n_queries.items():
    workload_config={}
    for nd in nds:
        for alias in baselines:
#             alias=baseline['alias']

            baseline = alias
            baseline_path = result_paths[baseline]
            workload_config[baseline] = baseline_path
    vconfig[workload]=workload_config

print(vconfig)
    
queries, bp_verdict = load_and_merge(vconfig['random'])


# random_analyzed = defaultdict(dict)
# for k in bp.keys():
#     # SUM query only
#     _, RE_results, _ = RelativeError(queries[k], bp_pass[k]).analyze()
#     random_analyzed[k]={"Relative Error": RE_results
                        
#                        }

# taxi_nd={}
# taxi_nd['Random Queries']=random_analyzed


{'random': {'intel': 'taxi-verdict-intel.json', 'insta': 'taxi-verdict-insta.json', 'taxi': 'taxi-verdict-taxi.json'}}


In [12]:
print(np.percentile(bp_verdict['intel']['intel'], 50))

0.9072935510339123


In [13]:
print(np.percentile(bp_verdict['insta']['insta'], 50))

0.9071375270105285


In [14]:
print(np.percentile(bp_verdict['taxi']['taxi'], 50))

0.9065338342816707
